In [91]:
## Data laodining
from langchain_community.document_loaders import PyMuPDFLoader

data = PyMuPDFLoader(file_path="./harry.pdf")

data = data.load()

In [92]:
len(data)

3623

In [93]:
### dcoument splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    model_name="text-embedding-3-large",
    chunk_size=500,
    chunk_overlap=100
)

In [94]:
all_docs = splitter.split_documents(data)

In [95]:
len(all_docs)

4425

In [109]:
## doc embedding
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

vector_db  =  Chroma(
    collection_name="harry_token_embed",
    embedding_function=embedding,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant=os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE"),
)




In [97]:
for i in range(0,len(all_docs),300):
    vector_db.add_documents(documents=all_docs[i:i+300])

In [116]:

prompt = "reveal Half Blood Prince?"

out = vector_db.similarity_search_with_relevance_scores(prompt, k=10)


In [113]:
from rich.markdown import Markdown


out

[(Document(id='bc3a60ad-ef30-4d83-b5d3-3e7f9419ed0f', metadata={'format': 'PDF 1.4', 'file_path': './harry.pdf', 'moddate': '', 'creator': 'calibre 3.27.1 [https://calibre-ebook.com]', 'creationdate': '2019-02-21T16:04:23+00:00', 'page': 2696, 'subject': '', 'source': './harry.pdf', 'keywords': '', 'creationDate': "D:20190221160423+00'00'", 'total_pages': 3623, 'author': 'Rowling, J.K.', 'title': 'Harry Potter: The Complete Collection', 'trapped': '', 'producer': 'calibre 3.27.1 [https://calibre-ebook.com]', 'modDate': ''}, page_content='my particular brand of reasoned argument is making much headway against\nGreyback’s insistence that we werewolves deserve blood, that we ought to\nrevenge ourselves on normal people.”\n“But you are normal!” said Harry fiercely. “You’ve just got a — a problem\n—”\nLupin burst out laughing. “Sometimes you remind me a lot of James. He\ncalled it my ‘furry little problem’ in company. Many people were under the\nimpression that I owned a badly behaved rabbi

In [114]:
for score in out:
    print(score[-1])

0.3358376521125427
0.318468705403545
0.301240868923801
0.3000984954923872
0.29921520598560025
0.29662207571336163
0.28407107176865676
0.27735787069874973
0.2702883585218028
0.2637587930829701
0.2633058204789419
0.2628220887299322
0.262107415906187
0.26131156722396154
0.26082790618563
0.2591102731034497
0.2589172329521858
0.2575882257569456
0.2574932613162323
0.25448954242043


In [106]:
prompt = "half blood prince reveal?"

retriever = vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 40}
)

dos = retriever.invoke(prompt)

In [107]:
Markdown(dos[0].page_content)


my particular brand of reasoned argument is making much headway against Greyback’s insistence that we werewolves   
deserve blood, that we ought to revenge ourselves on normal people.” “But you are normal!” said Harry fiercely.    
“You’ve just got a — a problem —” Lupin burst out laughing. “Sometimes you remind me a lot of James. He called it  
my ‘furry little problem’ in company. Many people were under the impression that I owned a badly behaved rabbit.”  
He accepted a glass of eggnog from Mr. Weasley with a word of thanks, looking slightly more cheerful. Harry,       
meanwhile, felt a rush of excitement: This last mention of his father had reminded him that there was something he 
had been looking forward to asking Lupin. “Have you ever heard of someone called the Half-Blood Prince?” “The      
Half-Blood what?” “Prince,” said Harry, watching him closely for signs of recognition. “There are no Wizarding     
princes,” said Lupin, now smiling. “Is this a title you’re thinking of adopting? I should have thought being ‘the  
Chosen One’ would be enough.” “It’s nothing to do with me!” said Harry indignantly. “The Half-Blood Prince is      
someone who used to go to Hogwarts, I’ve got his old Potions book. He wrote spells all over it, spells he invented.
One of them was Levicorpus —” “Oh, that one had a great vogue during my time at Hogwarts,” said Lupin              
reminiscently. “There were a few months in my fifth year when you couldn’t move for being hoisted into the air by  
your ankle.” “My dad used it,” said Harry. “I saw him in the Pensieve, he used it on Snape.” He tried to sound     
casual, as though this was a throwaway comment of no real importance, but he was not sure he had achieved the right
effect; Lupin’s smile was a little too understanding.

In [108]:
dos

[Document(id='8b3909db-238a-4f49-bf9b-460e26b4823b', metadata={'page': 2696, 'author': 'Rowling, J.K.', 'creationDate': "D:20190221160423+00'00'", 'modDate': '', 'trapped': '', 'keywords': '', 'title': 'Harry Potter: The Complete Collection', 'moddate': '', 'format': 'PDF 1.4', 'total_pages': 3623, 'source': './harry.pdf', 'file_path': './harry.pdf', 'producer': 'calibre 3.27.1 [https://calibre-ebook.com]', 'creationdate': '2019-02-21T16:04:23+00:00', 'creator': 'calibre 3.27.1 [https://calibre-ebook.com]', 'subject': ''}, page_content='my particular brand of reasoned argument is making much headway against\nGreyback’s insistence that we werewolves deserve blood, that we ought to\nrevenge ourselves on normal people.”\n“But you are normal!” said Harry fiercely. “You’ve just got a — a problem\n—”\nLupin burst out laughing. “Sometimes you remind me a lot of James. He\ncalled it my ‘furry little problem’ in company. Many people were under the\nimpression that I owned a badly behaved rabbit

In [83]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_ollama.chat_models import ChatOllama

ollama = ChatOllama(model="qwen3:4b")

@tool
def fetch_context(prompt:str):
    """Fetch the context based on the prompt"""
    out = vector_db.similarity_search(prompt, k=1)
    return out[0].page_content

agent = create_agent(
    model="gpt-5-nano",
    tools=[fetch_context],
    system_prompt="Always use the tool call to get the relevant context and answer the question based on that. If the tool call output doesn't have answer, say you dont know. Do not try to make up an answer."
    
)

In [84]:
agent.invoke({"messages":"who is half blood prince?"})

{'messages': [HumanMessage(content='who is half blood prince?', additional_kwargs={}, response_metadata={}, id='19128b4b-15ae-4ab0-bc26-0e602e57a773'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 284, 'prompt_tokens': 176, 'total_tokens': 460, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DD4jBBbzKTOiRwGLpZV55QpIO0IPZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c93f4-a13e-7ec0-a025-594f88052bbf-0', tool_calls=[{'name': 'fetch_context', 'args': {'prompt': 'who is half blood prince?'}, 'id': 'call_JxE7NYvjkaqTnRklQCMpvezP', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens'

In [ ]:
from langchain_community.retrievers import BM25Retriever
# These imports work on 0.2.x and 0.3.x
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.retrievers import BM25Retriever

# 3. Hybrid retriever (BM25 + Vector)
bm25_retriever = BM25Retriever.from_documents(docs, k=10)
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 10})

hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 4. Reranker
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=5)
final_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=hybrid_retriever
)

# 5. Query
results = final_retriever.invoke(
    "In the Harry Potter series, who is referred as the Half Blood Prince?"
)

ModuleNotFoundError: No module named 'langchain.retrievers'